<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Predictive_Analytics_with_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torch.optim as optim

# Load MNIST dataset (MNIST is commonly used for handwritten digit recognition)
mnist_dataset = torchvision.datasets.MNIST(root='.', download=True, transform=torchvision.transforms.ToTensor())

# Load MNIST images and labels
images, mnist_labels = mnist_dataset.data, mnist_dataset.targets

# Define device and dataloader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loader = torch.utils.data.DataLoader(mnist_dataset, batch_size=32, shuffle=True)

# Define encoder architecture
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 12),
            torch.nn.ReLU()
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(12, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28 * 28),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

autoencoder = Autoencoder().to(device)

# Train the autoencoder
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    for data in loader:
        img, _ = data
        img = img.view(img.size(0), -1).to(device).float()  # Convert to float
        output = autoencoder(img)
        loss = criterion(output, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

# Save the trained autoencoder model
torch.save(autoencoder.state_dict(), "autoencoder.pth")

# Encode and reconstruct images
encoded_images = []
reconstructed_images = []
autoencoder.eval()
with torch.no_grad():
    for img, label in zip(images, mnist_labels):
        img = img.view(-1, 28 * 28).to(device).float()  # Convert to float
        encoded_img = autoencoder.encoder(img)
        reconstructed_img = autoencoder.decoder(encoded_img)
        encoded_images.append(encoded_img.cpu())
        reconstructed_images.append(reconstructed_img.cpu())

# Compare decoded images with originals
original_images = images.view(-1, 28 * 28).float()  # Convert to float
difference = []

for orig, dec in zip(original_images, reconstructed_images):
    diff = torch.sum(torch.abs(orig - dec))
    difference.append(diff.item())

print(f"Average difference: {sum(difference) / len(original_images):.4f}")